In [3]:
import torch
from dataset.new_chlora import ChloraData
import os
import numpy as np
from utils import cal_stat, cal_region_stat
from torch.utils.data import DataLoader
import pandas as pd


In [ ]:
# Define data loader
ds_base  = "D:/ceeres/03_Program/01_SYSUM/00_science/04_image_recovery/01_program/00_dataset/"
root_dir = os.path.join(ds_base, "D:/ceeres/03_Program/01_SYSUM/00_science/04_image_recovery/01_program/00_dataset/chlora_month")
mask_dir = os.path.join(ds_base, "D:/ceeres/03_Program/01_SYSUM/00_science/04_image_recovery/01_program/00_dataset/chl_mask")

code_dir  = "D:/ceeres/03_Program/01_SYSUM/00_science/04_image_recovery/01_program/git_repo/convlstm_en_de/"
stats_dir = os.path.join(code_dir, "record/stats/")
img_dir   = os.path.join(code_dir, "record/img/")
log_dir   = os.path.join(code_dir, "record/log/")

results   = os.path.join(code_dir, "test/20230315-ED/")
os.makedirs(results,exist_ok=True)

frames      = 4
shape_scale = [480,480]

train_dataset = ChloraData(root_dir, frames, shape_scale, is_month=True, mask_dir=mask_dir, is_train=True,is_shuffle=False)
train_loader  = DataLoader(train_dataset, batch_size=1, shuffle=False)
test_dataset = ChloraData(root_dir, frames, shape_scale, is_month=True, mask_dir=mask_dir, is_train=False,is_shuffle=False)
test_loader  = DataLoader(test_dataset, batch_size=1, shuffle=False)

In [ ]:
len(train_dataset.chunk_list)+len(test_dataset.chunk_list)

In [ ]:
def list2csv(lists, csv_name):
    df = pd.DataFrame(lists)
    df.to_csv(csv_name, index=False, header=False)
csv_name = "D:/ceeres/03_Program/01_SYSUM/00_science/04_image_recovery/01_program/git_repo/convlstm_en_de/test/20230324-ED/20230324-ED.csv"
list2csv(train_dataset.chunk_list+test_dataset.chunk_list, csv_name)

In [4]:
dir_ds = "D:/ceeres/03_Program/01_SYSUM/00_science/04_image_recovery/01_program/git_repo/convlstm_en_de/test/20230324-ED/"
len(os.listdir(dir_ds))

250

In [5]:
dir_list = [os.path.join(dir_ds, i) for i in os.listdir(dir_ds) if i.split(".")[-1] == "npy"]


In [6]:
train_stat_list = []
label_stat_list = []
pred_stat_list  = []

for file in dir_list:
    array = np.load(file)
    train_stat_list.append(cal_stat(array[0],file))
    label_stat_list.append(cal_stat(array[1],file))
    pred_stat_list.append(cal_stat(array[2],file))

save_dir = "D:/ceeres/03_Program/01_SYSUM/00_science/04_image_recovery/01_program/git_repo/convlstm_en_de/test/20230324-ED/"
np.save(os.path.join(save_dir, "train_stat.npy"), np.array(train_stat_list))
np.save(os.path.join(save_dir, "label_stat.npy"), np.array(label_stat_list))
np.save(os.path.join(save_dir, "pred_stat.npy"), np.array(pred_stat_list))

In [ ]:
import arcpy
import os
# 定义输出栅格文件路径和文件名
base_dir = ""
for i in os.listdir(base_dir):
    file_dir = os.path.join(base_dir, i)
    if file_dir.split(".")[-1] == "shp":
        input_raster = file_dir
        output_raster = file_dir.replace(".shp", ".tif")
        cell_size = arcpy.Point(0.0001, 0.0001)
out_raster = "C:/data/out.tif"

# 定义输出栅格的左下角坐标
lower_left = arcpy.Point(-180, -90)

# 定义输出栅格的像素大小，x方向分辨率为0.01度，y方向分辨率为0.01度
cell_size = arcpy.Point(0.01, 0.01)

# 定义输出栅格的宽度和高度
num_rows = 1000
num_cols = 1000

# 创建输出栅格
arcpy.CreateRasterDataset_management(
    out_path="C:/data", 
    out_name="out.tif",
    raster_type="TIFF",
    number_of_bands=1,
    pixel_type="32_BIT_FLOAT",
    width=num_cols, 
    height=num_rows, 
    pixel_width=cell_size.X, 
    pixel_height=cell_size.Y, 
    spatial_reference=arcpy.SpatialReference(4326),  # 使用WGS84投影
    config_keyword="",
    tile_size="128 128",
    pyramid_level="-1",
    compression="LZ77",
    compression_quality="100",
    nodata_value=None,
    pyramids="PYRAMIDS -1 NEAREST DEFAULT",
    stats="STATS -1 0.5",
    keywords="")
